In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
from tqdm import tqdm

# Collect Data

## Oil Price

In [20]:
tickers_names = { "CL=F": "Crude Oil",
                  "HO=F": "Heating Oil",
                  "NG=F": "Natural Gas",
                  "RB=F": "RBOB Gasoline",
                  "BZ=F": "Brent Crude Oil"}

In [21]:
def FetchOilData(ticker_symbol, commodity_name):
    ticker = yf.Ticker(ticker_symbol)
    today = datetime.date.today().strftime('%Y-%m-%d')
    data = ticker.history(start="2000-01-01", end=today)
    
    # Reset the index and keep 'Date' as a column
    data.reset_index(inplace=True)
    
    # Ensure the 'Date' column is in the correct format
    if pd.api.types.is_datetime64_any_dtype(data['Date']):
        data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
    
    # Drop unnecessary columns
    data.drop(columns=['Dividends', 'Stock Splits'], inplace=True, errors='ignore')
    
    # Add additional columns for ticker and commodity
    data['ticker'] = ticker_symbol  
    data['commodity'] = commodity_name  
    
    # Convert all column names to lowercase for consistency
    data.columns = [col.lower() for col in data.columns]
    
    # Reorder columns to place 'ticker', 'commodity', and 'date' at the beginning
    data = data[['ticker', 'commodity', 'date'] + [col for col in data.columns if col not in ['ticker', 'commodity', 'date']]]
    
    return data



# 2. Loop over each symbol and append data to a DataFrame
all_data = []
for symbol, name in tickers_names.items():
    all_data.append(FetchOilData(symbol, name))

# Concatenate all the individual datasets into one
oil_price_data = pd.concat(all_data, ignore_index=True)
oil_price_data.to_csv('oil_price_data.csv', index = False)


In [22]:
oil_price_data
# (USD per barrel)

,ticker,commodity,date,open,high,low,close,volume
0,CL=F,Crude Oil,2000-08-23,31.950001,32.799999,31.950001,32.049999,79385
1,CL=F,Crude Oil,2000-08-24,31.900000,32.240002,31.400000,31.629999,72978
2,CL=F,Crude Oil,2000-08-25,31.700001,32.099998,31.320000,32.049999,44601
3,CL=F,Crude Oil,2000-08-28,32.040001,32.919998,31.860001,32.869999,46770
4,CL=F,Crude Oil,2000-08-29,32.820000,33.029999,32.560001,32.720001,49131
...,...,...,...,...,...,...,...,...
28784,BZ=F,Brent Crude Oil,2025-01-09,76.160004,77.269997,75.699997,76.919998,31706
28785,BZ=F,Brent Crude Oil,2025-01-10,77.209999,80.760002,77.040001,79.760002,92049
28786,BZ=F,Brent Crude Oil,2025-01-13,79.550003,81.669998,79.540001,81.010002,77109
28787,BZ=F,Brent Crude Oil,2025-01-14,80.910004,81.139999,79.860001,79.919998,67129


## Stock Price

In [23]:
tickers_names = {"SHEL": "Shell",
                 "XOM": "ExxonMobil",
                 "BP": "BP"}


In [24]:
def FetchStockData(ticker_symbol, company_name):
    ticker = yf.Ticker(ticker_symbol)
    today = datetime.date.today().strftime('%Y-%m-%d')
    data = ticker.history(start="2000-01-01", end=today)
    
    # Reset the index and keep 'Date' as a column
    data.reset_index(inplace=True)
    
    # Ensure the 'Date' column is in the correct format
    if pd.api.types.is_datetime64_any_dtype(data['Date']):
        data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
    
    # Drop unnecessary columns
    data.drop(columns=['Dividends', 'Stock Splits'], inplace=True, errors='ignore')
    
    # Add additional columns for ticker and commodity
    data['ticker'] = ticker_symbol  
    data['company'] = company_name  
    
    # Convert all column names to lowercase for consistency
    data.columns = [col.lower() for col in data.columns]
    
    # Reorder columns to place 'ticker', 'commodity', and 'date' at the beginning
    data = data[['ticker', 'company', 'date'] + [col for col in data.columns if col not in ['ticker', 'company', 'date']]]
    
    return data



# 2. Loop over each symbol and append data to a DataFrame
all_data = []
for symbol, name in tickers_names.items():
    all_data.append(FetchStockData(symbol, name))

# Concatenate all the individual datasets into one
stock_data = pd.concat(all_data, ignore_index=True)
stock_data.to_csv('stock_data.csv', index = False)


In [25]:
stock_data

,ticker,company,date,open,high,low,close,volume
0,SHEL,Shell,2000-01-03,18.295866,18.584902,18.199726,18.295866,1999300
1,SHEL,Shell,2000-01-04,17.756624,18.276766,17.699002,17.756624,2131900
2,SHEL,Shell,2000-01-05,18.411417,18.604005,18.180311,18.411417,2861900
3,SHEL,Shell,2000-01-06,18.796589,18.835106,18.449930,18.796589,2430300
4,SHEL,Shell,2000-01-07,19.432287,19.547840,19.181769,19.432287,2838200
...,...,...,...,...,...,...,...,...
18895,BP,BP,2025-01-13,31.100000,31.549999,31.049999,31.219999,9705500
18896,BP,BP,2025-01-14,30.600000,31.200001,30.570000,31.090000,13758900
18897,BP,BP,2025-01-15,31.219999,31.370001,30.980000,31.299999,10964600
18898,BP,BP,2025-01-16,31.469999,31.860001,31.389999,31.780001,11435600


# EDA

## Preparing the Data
Objective: Clean and structure the data for time-series analysis.

Action:
- Convert the date column in both datasets to datetime format.
- Sort the data by date to ensure chronological order.
- Handle missing values, either by interpolation or forward/backward filling.
- Aggregate data if needed (e.g., daily to monthly averages for smoother trends).

In [ ]:
# Transform date column to datetime
oil_price_data['date'] = pd.to_datetime(oil_price_data['date'])
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Check the date range
earliest_dates = []
newest_dates = []
tickers_names = { "CL=F": "Crude Oil",
                  "HO=F": "Heating Oil",
                  "NG=F": "Natural Gas",
                  "RB=F": "RBOB Gasoline",
                  "BZ=F": "Brent Crude Oil"}
for t in list(tickers_names.keys()):
    data = oil_price_data[oil_price_data['ticker'] == t]
    earliest_date = data['date'].min()
    newest_date = data['date'].max()
    earliest_dates.append(earliest_date)
    newest_dates.append(newest_date)
    # print(f"{t}: {earliest_date}, {newest_date}")

tickers_names = {"SHEL": "Shell",
                 "XOM": "ExxonMobil",
                 "BP": "BP"}
for t in list(tickers_names.keys()):
    data = stock_data[stock_data['ticker'] == t]
    earliest_date = data['date'].min()
    newest_date = data['date'].max()
    earliest_dates.append(earliest_date)
    newest_dates.append(newest_date)
    # print(f"{t}: {earliest_date}, {newest_date}")

# Adjust the date range of the two df

start_date = max(earliest_dates)
end_date = min(newest_dates)
oil_price_data = oil_price_data[(oil_price_data['date'] >= start_date) & (oil_price_data['date'] <= end_date)]
stock_data = stock_data[(stock_data['date'] >= start_date) & (stock_data['date'] <= end_date)]

In [32]:
print(stock_data['ticker'].value_counts())

ticker
SHEL    4396
XOM     4396
BP      4396
Name: count, dtype: int64


In [29]:
oil_price_data

,ticker,commodity,date,open,high,low,close,volume
1730,CL=F,Crude Oil,2007-07-30,76.949997,77.330002,76.050003,76.830002,189456
1731,CL=F,Crude Oil,2007-07-31,76.699997,78.279999,76.599998,78.209999,196464
1732,CL=F,Crude Oil,2007-08-01,77.940002,78.769997,76.089996,76.529999,306683
1733,CL=F,Crude Oil,2007-08-02,76.739998,77.440002,75.519997,76.860001,214006
1734,CL=F,Crude Oil,2007-08-03,76.889999,77.360001,75.040001,75.480003,233021
...,...,...,...,...,...,...,...,...
28784,BZ=F,Brent Crude Oil,2025-01-09,76.160004,77.269997,75.699997,76.919998,31706
28785,BZ=F,Brent Crude Oil,2025-01-10,77.209999,80.760002,77.040001,79.760002,92049
28786,BZ=F,Brent Crude Oil,2025-01-13,79.550003,81.669998,79.540001,81.010002,77109
28787,BZ=F,Brent Crude Oil,2025-01-14,80.910004,81.139999,79.860001,79.919998,67129


## Exploratory Analysis
Objective: Understand the data trends and patterns over time.

Action:
- Plot time-series data for both oil prices and stock prices.
- Identify seasonality (e.g., monthly or yearly trends).
- Look for anomalies or outliers in the data

```python
# Code Example
import matlotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(stock_data['date'], stock_data['close'], label='Stock Prices')
plt.title('Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
```

## Decomposing the Time Series
Objective: Break the series into trend, seasonality, and residual components.

Action:
- Use statsmodels to decompose the series.
- Visualize components to understand the underlying structure

```python
# Example code
from statsmodels.tsa.seasonal import seasonal_decompose

# Decompose time series
decomposition = seasonal_decompose(stock_data['close'], model='additive', period=365)
decomposition.plot()
plt.show()
```

## Stationarity Check
Objective: Verify if the time series is stationary (a requirement for some models like ARIMA).

Action:
- Use the Augmented Dickey-Fuller (ADF) test to check stationarity.
- If non-stationary, apply transformations like differencing or log transformations.

```python
# Example code
from statsmodels.tsa.stattools import adfuller

result = adfuller(stock_data['close'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])
```

# ML Models

## Time-Series Modeling
Objective: Build models to forecast stock prices based on past data.

Action:
- Start with ARIMA (Auto-Regressive Integrated Moving Average).
- Experiment with more advanced models like Prophet for better interpretability.
- Use oil price data as an external regressor in models like SARIMAX.

```python
# Code Example for ARIMA
from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA model
model = ARIMA(stock_data['close'], order=(1, 1, 1))
results = model.fit()
print(results.summary())

# Plot forecast
results.plot_predict(dynamic=False)
plt.show()
```

## Evaluation
Objective: Measure the accuracy of your model.

Action:
- Use metrics like Mean Absolute Error (MAE) or Root Mean Squared Error (RMSE).
- Compare forecasted values against actual data

```python
# Code Example
from sklearn.metrics import mean_squared_error
import numpy as np

forecast = results.predict(start=some_start_date, end=some_end_date)
rmse = np.sqrt(mean_squared_error(actual_data, forecast))
print('RMSE:', rmse)
```

# Using the trends of oil price data to predict stock price

Using the trends of oil price data to predict stock price trends involves combining time-series forecasting with regression modeling. Here’s a structured plan for achieving this:

---

### **1. Data Preparation**
#### **Objective**: Ensure both datasets are clean, synchronized, and ready for modeling.
- **Steps**:
  1. **Align Date Ranges**:
     - Filter both datasets to have the same date range to ensure consistency.
  2. **Aggregate Data** (if needed):
     - If stock and oil price data are at different frequencies (e.g., daily vs. weekly), aggregate them to a common frequency.
     - Use techniques like averaging, taking the last value of the period, or summing (for volume).
  3. **Feature Creation**:
     - Create lagged features for oil prices (e.g., oil price of the last day, week, or month).
     - Calculate rolling averages or moving averages of oil prices and stock prices to smooth out noise.
     - Add volatility metrics for oil prices (e.g., rolling standard deviation).
  4. **Merge Datasets**:
     - Combine stock and oil price datasets using the `date` column as the key.

### **Key Adjustments for Your Project**
- **Data Normalization**: Since oil prices and stock prices may have different scales, normalize the data to avoid skewed models.
- **Stationarity**: Address non-stationarity in both datasets before modeling.
- **Exogenous Variables**: Consider adding macroeconomic indicators (e.g., inflation, interest rates) to the model if available.

### **2. Exploratory Analysis**
#### **Objective**: Understand the relationship between oil prices and stock prices.
- **Steps**:
  1. **Visualize Trends**:
     - Plot oil price trends alongside stock price trends to observe correlations.
  2. **Correlation Analysis**:
     - Calculate and visualize correlations between oil prices and stock prices using a heatmap.
  3. **Stationarity Check**:
     - Check stationarity of both datasets using the Augmented Dickey-Fuller (ADF) test.
  4. **Granger Causality Test**:
     - Perform Granger causality testing to check if past oil prices can predict stock prices.


### **3. Modeling**
#### **Objective**: Build predictive models that use oil price trends to forecast stock price trends.
#### **Option 1: Time-Series Regression (SARIMAX)**
   - **Steps**:
     1. Use a **SARIMAX** (Seasonal Auto-Regressive Integrated Moving Average with Exogenous Variables) model.
     2. Set stock prices as the dependent variable and oil prices (and lagged features) as exogenous variables.
   - **Example**:
     ```python
     from statsmodels.tsa.statespace.sarimax import SARIMAX

     # Define the SARIMAX model
     model = SARIMAX(stock_data['close'], 
                     exog=oil_price_data[['close']], 
                     order=(1, 1, 1), 
                     seasonal_order=(0, 0, 0, 0))
     results = model.fit()
     print(results.summary())
     results.plot_diagnostics(figsize=(12, 8))
     ```

#### **Option 2: Machine Learning**
   - **Steps**:
     1. Split the data into training and testing sets.
     2. Use oil price features (e.g., `close`, rolling averages, lagged features) as predictors.
     3. Train models such as:
        - Random Forest
        - Gradient Boosting (e.g., XGBoost, LightGBM)
        - Neural Networks
     4. Evaluate using metrics like RMSE and R².
   - **Example**:
     ```python
     from sklearn.ensemble import RandomForestRegressor
     from sklearn.metrics import mean_squared_error
     import numpy as np

     # Define predictors and target
     X = oil_price_data[['close', 'lag_1', 'rolling_mean_7']]
     y = stock_data['close']

     # Train-test split
     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

     # Train Random Forest
     model = RandomForestRegressor(n_estimators=100, random_state=42)
     model.fit(X_train, y_train)

     # Evaluate
     predictions = model.predict(X_test)
     print("RMSE:", np.sqrt(mean_squared_error(y_test, predictions)))
     ```

#### **Option 3: Deep Learning**
   - Use LSTMs or GRUs, which are specifically designed for sequential data.
   - Feed oil price trends and lagged stock prices as inputs.

### **4. Model Evaluation**
#### **Objective**: Assess model performance and refine as needed.
- **Steps**:
  1. Compare predicted trends with actual trends using plots.
  2. Calculate evaluation metrics:
     - Mean Absolute Error (MAE)
     - Root Mean Squared Error (RMSE)
     - \( R^2 \)
  3. Tune hyperparameters to improve accuracy.


### **5. Insights and Visualization**
#### **Objective**: Present findings effectively.
- **Steps**:
  1. Create interactive visualizations (e.g., using `plotly` or `dash`) to show:
     - Oil price trends
     - Stock price trends
     - Predictions vs. actual data
  2. Summarize key insights, such as the impact of oil price changes on stock prices.
